<a href="https://colab.research.google.com/github/celobastos/AnaliseVeicular/blob/master/AnaliseVeicular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install ultralytics opencv-python-headless pandas numpy scipy
import cv2
import torch
import pandas as pd
import numpy as np
from collections import defaultdict

In [14]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [2, 3, 5, 7]

VIDEO_PATHS = [
    'istockphoto-1429837312-640_adpp_is.mp4',
    'istockphoto-1351238273-640_adpp_is.mp4'
]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-25 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [15]:
def detect_traffic_light_state(frame):
    """
    Retorna True se o semáforo estiver VERDE, False se VERMELHO.
    Usa segmentação de cor em HSV num ROI fixo.
    """
    h, w = frame.shape[:2]
    roi = frame[0:int(h*0.2), int(w*0.8):w]

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, (40, 50, 50), (90, 255, 255))
    mask_red1  = cv2.inRange(hsv, (0, 70, 50), (10, 255, 255))
    mask_red2  = cv2.inRange(hsv, (170, 70, 50), (180, 255, 255))

    green_area = cv2.countNonZero(mask_green)
    red_area   = cv2.countNonZero(mask_red1) + cv2.countNonZero(mask_red2)

    return green_area > red_area

In [16]:
from scipy.spatial import distance as dist

class CentroidTracker:
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = {}       # objectID → (cX, cY)
        self.disappeared = {}   # objectID → frames sem detecção
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        """
        rects: lista de [x1,y1,x2,y2]
        Retorna dict {objectID: centroid}
        """
        if len(rects) == 0:
            for oid in list(self.disappeared):
                self.disappeared[oid] += 1
                if self.disappeared[oid] > self.maxDisappeared:
                    self.deregister(oid)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for i, (x1, y1, x2, y2) in enumerate(rects):
            inputCentroids[i] = (int((x1+x2)/2), int((y1+y2)/2))

        if len(self.objects) == 0:
            for c in inputCentroids:
                self.register(tuple(c))
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # Matriz de distâncias
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # Associações: menor distância primeiro
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows, usedCols = set(), set()

            for row, col in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                if D[row, col] > 50:  # ajuste o limiar conforme seu cenário
                    continue
                oid = objectIDs[row]
                self.objects[oid] = tuple(inputCentroids[col])
                self.disappeared[oid] = 0
                usedRows.add(row)
                usedCols.add(col)

            # Detecta objetos desaparecidos e novos
            unusedRows = set(range(D.shape[0])) - usedRows
            unusedCols = set(range(D.shape[1])) - usedCols

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    oid = objectIDs[row]
                    self.disappeared[oid] += 1
                    if self.disappeared[oid] > self.maxDisappeared:
                        self.deregister(oid)
            else:
                for col in unusedCols:
                    self.register(tuple(inputCentroids[col]))

        return self.objects


In [17]:
def process_video_centroid(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    sec_interval = int(fps)
    ct = CentroidTracker(maxDisappeared=40)

    # Define linha em 60% da altura
    _, sample = cap.read()
    h, w = sample.shape[:2]
    line_y = int(h * 0.6)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    total = 0
    by_second = defaultdict(int)
    green_count = 0
    red_count = 0
    frame_idx = 0
    prev_centroids = {}

    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1

        # 1) detecta CARRos
        results = model(frame[..., ::-1], size=640)
        dets = results.xyxy[0].cpu().numpy()
        rects = [[int(x1),int(y1),int(x2),int(y2)] for x1,y1,x2,y2,conf,cls in dets]

        # 2) rastreia por centroide
        objects = ct.update(rects)

        # 3) contagem
        current_sec = frame_idx // sec_interval
        is_green = detect_traffic_light_state(frame)

        for oid, centroid in objects.items():
            if oid in prev_centroids:
                prev_cy = prev_centroids[oid][1]
                if prev_cy < line_y <= centroid[1]:
                    total += 1
                    by_second[current_sec] += 1
                    if is_green: green_count += 1
                    else:        red_count += 1

        prev_centroids = objects.copy()

    cap.release()

    # Salva CSV de distribuição por segundo
    df = pd.DataFrame([
        {'second': s, 'count': by_second[s]}
        for s in sorted(by_second)
    ])
    base = video_path.rsplit('.', 1)[0]
    df.to_csv(f'{base}_time_dist.csv', index=False)

    # Salva resumo em TXT
    with open(f'{base}_summary.txt', 'w') as f:
        f.write(f'Vídeo: {video_path}\n')
        f.write(f'Total veículos: {total}\n')
        f.write(f'Durante VERDE: {green_count}\n')
        f.write(f'Durante VERMELHO: {red_count}\n')

    print(f'Relatório gerado para {base}')


In [18]:
for path in VIDEO_PATHS:
    process_video_centroid(path)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

Relatório gerado para istockphoto-1429837312-640_adpp_is


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

Relatório gerado para istockphoto-1351238273-640_adpp_is


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am